In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output, Markdown
import ipywidgets as widgets
import time

robot = Robot()
camera = Camera.instance()

import dt_apriltags

detector = dt_apriltags.Detector()

# TODO: Update these values below
cam_mtx = [?, ?, ?, ?]
AT_size = ?

# Creating Widgets for displaying output
image_widget = widgets.Image(format='jpeg')
tag_label = widgets.Label()
pose_t_output = widgets.Output()
pose_R_output = widgets.Output()

display(widgets.HBox([image_widget, tag_label, pose_t_output, pose_R_output]))

while True:
    robot.forward(0.2)
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # TODO: Write this line with the necessary parameters from the function header shown above
    detections = detector.detect(gray_frame, True, cam_mtx, AT_size)

    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    image_widget.value = bgr8_to_jpeg(image)

    # Getting Pose Values from AprilTag
    if detections:
        tag_label.value = f"**Last Detected ID:** {tag_id}"
        pose_t_str = np.array2string(detections[0].pose_t, precision=3, suppress_small=True)
        pose_R_str = np.array2string(detections[0].pose_R, precision=3, suppress_small=True)
        with pose_t_output:
            pose_t_output.clear_output(wait=True)
            print("Translation Pose (t):")
            print(pose_t_str)
        with pose_R_output:
            pose_R_output.clear_output(wait=True)
            print("Rotation Pose (R):")
            print(pose_R_str)
        pose_t = detection[0].pose_t
        z_distance = pose_t[2][0]
        if z_distance <= 0.1:
            robot.stop()
            break
        
        
    time.sleep(0.05)